In [0]:
# function to save matrix to a textfile
import numpy 

def save_matrix(name, matrix):
  numpy.savetxt(name, matrix, delimiter = ',')  

In [0]:
# define fourier transform (DFT) matrix

import numpy as np
def DFT_matrix(N):
    x, a = np.meshgrid(np.arange(N), np.arange(N))
    omega = np.exp(-2 * np.pi * 1j/N)
    W = np.power(omega, x * a ) / np.sqrt(N)
    return W

In [0]:
import numpy as np

# declare DFT matrix of size 8x8
dftmtx = DFT_matrix(8)
save_matrix("dftmatrix8.txt", dftmtx)

# declare DFT matrix of size 4x4
dftmtx4 = DFT_matrix(4)
save_matrix("dftmatrix4.txt", dftmtx4)

# conjugate transpose of DFT matrix of size 8x8
dftmtxh = np.transpose(np.conj(DFT_matrix(8)))

# conjugate transpose of DFT matrix of size 4x4
dftmtx4h = np.transpose(np.conj(DFT_matrix(4)))

In [0]:
# function to convert from PIL image to OpenCV for saving purposes

def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2: 
        pass
    elif new_image.shape[2] == 3:
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

In [0]:
# convert image into grayscale, then applying DFT transform
import cv2

image = cv2.imread('image.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite('gray_image.jpg', gray_image)
block_img = np.zeros(image.shape)
im_h, im_w = image.shape[:2]
bl_h, bl_w = 8, 8

for row in np.arange(im_h - bl_h + 1, step=bl_h):
    for col in np.arange(im_w - bl_w + 1, step=bl_w):
        block_img[row:row+bl_h, col:col+bl_w] = dftmtx.dot(image[row:row+bl_h, col:col+bl_w])

cv2.imwrite("dft_image.jpg", block_img)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


True

In [0]:
import numpy as np

# define and save matrices S and I for subsampling and interpolation respectively
s = np.asarray([[1/2, 1/2, 0, 0, 0, 0, 0, 0], [0, 0, 1/2, 1/2, 0, 0, 0, 0], [0, 0, 0, 0, 1/2, 1/2, 0, 0], [0, 0, 0, 0, 0, 0, 1/2, 1/2]])
save_matrix("s.txt", s)
i = np.asarray([[1, 0, 0, 0], [1/2, 1/2, 0, 0], [0, 1, 0, 0], [0, 1/2, 1/2, 0], [0, 0, 1, 0], [0, 0, 1/2, 1/2], [0, 0, 0, 1], [0, 0, 0, 1]])
save_matrix("i.txt", i)

# define S^T and I^T
st = np.transpose(s)
save_matrix("st.txt", st)
it = np.transpose(i)
save_matrix("it.txt", it)

In [0]:
import numpy as np
import skimage.measure

image = cv2.imread("image.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
inverse_dft_interp = np.zeros(image.shape)
inverse_dft_samp = np.zeros(image.shape)
sample_space = np.zeros(image.shape)
interp_space = np.zeros(image.shape)
sample_freq = np.zeros(image.shape)
interp_freq = np.zeros(image.shape)
im_h, im_w = image.shape[:2]
bl_h, bl_w = 8, 8
sm_h, sm_w = 4, 4

# perform subsampling and interpolation for spatial domain
# subsample gray image, f_4 (spatial)
gray_image = cv2.imread("image.jpg")
gray_image = cv2.cvtColor(gray_image, cv2.COLOR_BGR2GRAY)
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):        
        sample_space[row:row+sm_h, col:col+sm_w] = s.dot(gray_image[row:row+bl_h, col:col+bl_w]).dot(st)
cv2.imwrite("sampled_spatial.jpg", sample_space)

# interpolate sampled image, f_8 (spatial)
sampspace = cv2.imread("sampled_spatial.jpg")
sampspace = cv2.cvtColor(sampspace, cv2.COLOR_BGR2GRAY)
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):        
        interp_space[row:row+bl_h, col:col+bl_w] = i.dot(sampspace[row:row+sm_h, col:col+sm_w]).dot(it)
cv2.imwrite("interpolated_spatial.jpg", interp_space)   

# perform subsampling and interpolation for frequency domain
# subsample dft image, F_4 (frequency)   
dft_image = cv2.imread("dft_image.jpg")
dft_image = cv2.cvtColor(dft_image, cv2.COLOR_BGR2GRAY)
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):
        sample_freq[row:row+sm_h, col:col+sm_w] = dftmtx4.dot(s).dot(dftmtxh).dot(dft_image[row:row+bl_h, col:col+bl_w]).dot(dftmtxh).dot(st).dot(dftmtx4)
cv2.imwrite("sampled_frequency.jpg", sample_freq)

# interpolate sampled image, F_8 (frequency)
sampfreq = cv2.imread("sampled_frequency.jpg")
sampfreq = cv2.cvtColor(sampfreq, cv2.COLOR_BGR2GRAY)
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):
        interp_freq[row:row+bl_h, col:col+bl_w] = dftmtx.dot(i).dot(dftmtx4h).dot(sampfreq[row:row+sm_h, col:col+sm_w]).dot(dftmtx4h).dot(it).dot(dftmtx)
cv2.imwrite("interpolated_frequency.jpg", interp_freq)    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: ComplexWarning: Casting complex values to real discards the imaginary part


True

In [0]:
# inverse DFT for sampled image in frequency domain
samplefreq = cv2.imread("sampled_frequency.jpg")
samplefreq = cv2.cvtColor(samplefreq, cv2.COLOR_BGR2GRAY)
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):        
        inverse_dft_samp[row:row+sm_h, col:col+sm_w] = samplefreq[row:row+sm_h, col:col+sm_h].dot(dftmtx4h)
cv2.imwrite("inverse_dft_samp.jpg", inverse_dft_samp)

# inverse DFT for final interpolated image in frequency domain
interpfreq = cv2.imread("sampled_frequency.jpg")
interpfreq = cv2.cvtColor(interpfreq, cv2.COLOR_BGR2GRAY)
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):        
        inverse_dft_interp[row:row+bl_h, col:col+bl_w] = interpfreq[row:row+bl_h, col:col+bl_w].dot(dftmtxh)
cv2.imwrite("inverse_dft_interp.jpg", inverse_dft_interp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: ComplexWarning: Casting complex values to real discards the imaginary part
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


True